## Translator Model - Fine Tune

This notebook trains / fine-tunes pre-trained  translation model according to the guide provided on the Hugging Face page.

    https://huggingface.co/docs/transformers/tasks/translation

**PyTorch used for finetuning.**

In [1]:
# Import Libraries
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

2022-10-13 16:19:48.589183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 16:19:59.055908: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Explore dataset

In [2]:
# Load data
books0 = load_dataset("opus_books", "en-fr")

Found cached dataset opus_books (/home/kyagci/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:

books0.keys()

dict_keys(['train'])

In [4]:
books0["train"][0]

{'id': '0', 'translation': {'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'}}

In [5]:
len(books0["train"])

127085

### Tokenize and train model

In [6]:
books = books0["train"].train_test_split(test_size=0.2)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=128)

In [11]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [12]:
tokenized_books = books.map(preprocess_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/kyagci/anaconda3/envs/base_env1/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 101668
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1589
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method 

Epoch,Training Loss,Validation Loss
1,1.951400,1.721285


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Copy vocab file to ./results/checkpoint-500/spiece.model
/home/kyagci/anaconda3/envs/base_env1/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint

TrainOutput(global_step=1589, training_loss=1.9952206092483973, metrics={'train_runtime': 409.5139, 'train_samples_per_second': 248.265, 'train_steps_per_second': 3.88, 'total_flos': 3190632216526848.0, 'train_loss': 1.9952206092483973, 'epoch': 1.0})

In [21]:
save_directory ='./translatorModel3'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Configuration saved in ./translatorModel3/config.json
Model weights saved in ./translatorModel3/pytorch_model.bin
tokenizer config file saved in ./translatorModel3/tokenizer_config.json
Special tokens file saved in ./translatorModel3/special_tokens_map.json
Copy vocab file to ./translatorModel3/spiece.model


('./translatorModel3/tokenizer_config.json',
 './translatorModel3/special_tokens_map.json',
 './translatorModel3/spiece.model',
 './translatorModel3/added_tokens.json',
 './translatorModel3/tokenizer.json')

In [4]:
# Run this cell if loading the model ONLY

#model_directory ='./translatorModel3'
#model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
#tokenizer = AutoTokenizer.from_pretrained(model_directory, model_max_length=128)

In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/kyagci/.huggingface/token


In [7]:
model_name = 't5small-finetuned-opusbooks-en-fr'
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

CommitInfo(commit_url='https://huggingface.co/kamileyagci/t5small-finetuned-opusbooks-en-fr/commit/ba3ba4f292ad7ed74d0687126483e6d0f5d7e01d', commit_message='Upload tokenizer', commit_description='', oid='ba3ba4f292ad7ed74d0687126483e6d0f5d7e01d', pr_url=None, pr_revision=None, pr_num=None)